In [1]:
import numpy as np
import pandas as pd
import os
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.neighbors import KNeighborsClassifier

from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import CategoricalNB

from sklearn.pipeline import make_pipeline
import tdm
import math
import string
from sklearn.metrics import confusion_matrix, accuracy_score ,ConfusionMatrixDisplay, f1_score

from sklearn import preprocessing
from sklearn.pipeline import Pipeline

import nltk
from nltk.tokenize import sent_tokenize , word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
import re

from sklearn.naive_bayes import GaussianNB, MultinomialNB
import numpy as np

Functions to lemmatise the text and to correct OCR mistakes

In [2]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')


def ptb_to_wordnet(PTT):

    if PTT.startswith('J'):
        ## Adjective
        return 'a'
    elif PTT.startswith('V'):
        ## Verb
        return 'v'
    elif PTT.startswith('N'):
        ## Noune
        return 'n'
    elif PTT.startswith('R'):
        ## Adverb
        return 'r'
    else:
        return ''



find_replace = pd.read_csv('find_replace.csv')

lemmatiser = WordNetLemmatizer()


def clean_text(text):
    text = str(text).lower().strip()
    text = re.sub( r'~' , 't' , text)
    text = re.sub( r'\'' , '' , text)
    text = re.sub( r'`' , '' , text)
    text = re.sub( r'\s+' , ' ' , text)
    text = re.sub( r'(\. )+' , '. ' , text)
    text = re.sub( r'-\s+' , '' , text)
    text = re.sub( r'\.\.' , '' , text)
    text = re.sub( r'tet\'ritorial' , 'territorial' , text)
    text = re.sub( r'\bwa3\b' , 'was' , text)
    text = re.sub( r'popu:" lation' , 'population' , text)
    text = re.sub( f'^rst' , 'first' , text )
    text = re.sub( f'9ov:ernment' , 'government' , text )
    text = re.sub( r'vio!at' , 'violat' , text)
    text = re.sub( r'\brilthts\b' , 'rights' , text)
    text = re.sub( r'\badantic\b' , 'atlantic' , text)
    text = re.sub( r'\bobbgatlons\b' , 'obligations' , text)
    text = re.sub( r'\bmination\b' , '' , text)
    text = re.sub( r'self-detert' , 'self-determination' , text)
    text = re.sub( r'orgamzatlon' , 'organization' , text)
    text = re.sub( r'pretident' , 'president' , text)
    text = re.sub( r'deh10cratic' , 'democratic' , text)
    text = re.sub( r'\bhau\b' , 'have' , text)
    text = re.sub( r'·' , ' ' , text  ) 
    text = re.sub( r'cha!\'ter' , 'chapter' , text)
    text = re.sub( r'(\')+' , '\'' , text)

    return text.strip()
    

def lemmatise(text):
    text = str(text).lower()

    #lemmatise the quote
    words = word_tokenize(text)
    pos = nltk.pos_tag(words)
    lemmatised = ''

    for i in range( 0 , len(words) ):

        posTag = ptb_to_wordnet( pos[i][1] )
        if re.search( r'\w+' , posTag , re.IGNORECASE ):
            lemma = lemmatiser.lemmatize( words[i] , posTag )
            if re.search( r'\w' , lemma ):
                lemmatised += ' '
            if lemma not in stop_words:
                lemmatised += lemma 
        else:
            lemma = lemmatiser.lemmatize( words[i] )
            if re.search( r'\w' , lemma ):
                lemmatised += ' '
            if lemma not in stop_words:
                lemmatised += lemma 
            
    return lemmatised.strip()


The directory named 'Quotations' contains all excel files with manual classifications of the quotations, taken from speeches delivered in between 1950 and 1979. 

The quotations are saved as `features` and the assigned codes are the `targets`. The quotations are lemmatised and cleaned using the methods defined above. 

In [3]:
features = []
targets = []

dir = 'Quotations'

files = []
rows = []

for file in os.listdir(dir):
    if (re.search(r'xlsx',file) and not(re.search( r'~',file))):
        path = os.path.join(dir,file)
        files.append(path)

for file in files:
    print(file)

    # Read excel
    df = pd.read_excel(file)

    # fragments
    for index,row in df.iterrows():
        quote = row['Quotation Content']
        quote = clean_text(quote)
        quote = lemmatise(quote)

        all_codes = row['Codes']
        lines = re.split( r'\n' , all_codes )
        code = ''
        country = '' 
        year = ''

        for line in lines:
            #print(line)
            if re.search( r'^s_' , line.strip() , re.IGNORECASE ):
                country = re.sub( r'^[Ss]_' , '' , line.upper() , re.IGNORECASE )
                
            if re.search( r'^y_' , line.strip() , re.IGNORECASE ):
                year = re.sub( r'^[Yy]_' , '' , line.upper() , re.IGNORECASE )
                
                if re.search( r'\(' , year): 
                    year = year[ :year.index('(') ].strip()
                year = int(year)

            
            if re.search( r'^m_' , line.strip()  ):
                code = line.lower()
                
        if code != 'm_specific multlilateral agreement' and code != 'm_quote' and len(code)>0 and quote != 'nan':

            ## inconsistent spellings
            if code == 'm_non intervention':
                code = 'm_non-intervention'

            elif code == 'm_non interference':
                code = 'm_non-interference'

            code = re.sub( r'[-]','_',code )
            code = re.sub( r'\s+' , '_' , code )

            features.append(quote)
            targets.append(code)

            rows.append( [file,country,year,quote,row['Quotation Content'],code] )


Quotations/List of quotations 1960-69.xlsx
Quotations/List of quotations 1955-59.xlsx
Quotations/List of quotations 1970-74.xlsx
Quotations/List of quotations 1975-79.xlsx
Quotations/List of quotations 1950-54.xlsx


The data from the Excel files is captured in a CSV file

In [4]:
columns = ['file_name', 'country', 'year', 'text', 'orig_text', 'category']
df_quotations = pd.DataFrame(rows,columns=columns)
df_quotations.to_csv('quotations_1950-1979.csv')

In [5]:
print( f'The excel files contain {df_quotations.shape[0]} quotations')

The excel files contain 4126 quotations


Create a training set and a test set, with a test set size of 30% 

In [6]:
for t in sorted(set(targets)):
    print(t)
    
# encoder = LabelEncoder()
# encoded_labels = encoder.fit(targets)
# labels_names = dict(zip(encoded_labels.classes_, encoded_labels.transform(encoded_labels.classes_)))
# targets = encoder.fit_transform(targets)

# print(targets)

m_cold_war
m_colonialism
m_general_reference_to_foreign_policy
m_historical_reference
m_non_interference
m_non_intervention
m_reference_to_state_leader
m_sovereignty_limited
m_specific_conflict
m_specific_multilateral_agreement
m_un_design
m_un_mandate
m_un_membership


Replace textual labels with numeric labels

In [7]:
features_train, features_test, targets_train , targets_test = train_test_split(
    features, targets, test_size=0.3, random_state=125
)

print(f'The training set contains {len(features_train)} quotations.')
print(f'The test set contains {len(features_test)} quotations.')

The training set contains 2888 quotations.
The test set contains 1238 quotations.


## K-nearest Neighbours

In [8]:
vec = TfidfVectorizer(binary=True, use_idf=True)

tfidf_train = vec.fit_transform(features_train).toarray()

knn = KNeighborsClassifier(n_neighbors=25)

knn_model = knn.fit( tfidf_train, targets_train )

# # Predict Output
predictions = knn_model.predict(tfidf_train)

print( confusion_matrix(targets_train, predictions  ) )
print( accuracy_score(targets_train, predictions) )

[[197   4  20   0   1   1   0  58  52   0   0   0   3]
 [ 18 102   5   0   0   0   0  61  89   0   0   2  11]
 [ 23   2  43   0   0   0   0  26  13   0   0   0   1]
 [  5   6   1   0   0   0   0  18  16   0   0   0   1]
 [ 25   1   6   0  14   0   0  50  47   0   0   0   1]
 [ 12   1   0   0   0   0   0  13  32   0   0   0   0]
 [  1   2   0   0   0   0  15   4   6   0   0   0   0]
 [ 26   7   9   0   2   0   0 485  68   0   0   1   1]
 [ 11  14  10   0   3   0   0  58 692   0   0   0   0]
 [ 17   2   3   0   1   0   0  35  25   5   0   0   2]
 [  3   0   2   0   0   0   0  32  11   0   2   1   4]
 [  4   8   2   0   2   0   0  70  66   0   0   4   5]
 [  9  13   2   0   0   0   1  28  29   0   0   1 103]]
0.5754847645429363


In [9]:

tfidf_test = vec.transform(features_test).toarray()

# Predict Output
predictions = knn_model.predict(tfidf_test)

print( confusion_matrix(targets_test , predictions  ) )
print( accuracy_score(targets_test ,predictions) )

[[ 70   4   7   0   1   0   0  22  39   0   0   0   2]
 [  4  39   1   0   1   0   0  31  48   0   0   0   6]
 [  8   3  18   0   1   0   0  18   9   0   0   0   3]
 [  4   2   0   0   0   0   0   8   6   0   0   0   0]
 [  9   1   3   0   2   0   0  16  12   0   0   0   0]
 [  5   1   0   0   2   0   0   7  13   0   0   0   0]
 [  0   0   0   0   0   0   7   2   4   0   0   0   0]
 [ 14   4   3   0   2   0   0 187  33   0   0   2   2]
 [ 11  11   3   0   1   0   0  20 271   0   0   0   0]
 [  9   0   1   0   1   0   0  13   7   0   0   0   0]
 [  1   0   0   0   0   0   0  27   5   0   0   1   2]
 [  4   3   3   0   0   0   0  36  35   0   0   1   2]
 [  3   7   2   0   0   0   0  17  17   0   0   0  38]]
0.5113085621970921


## Naive Bayes

Train a model based on Naive Bayes. 


In [10]:

vec = TfidfVectorizer(binary=True, use_idf=True)
tfidf_train = vec.fit_transform(features_train).toarray()


# Build a Gaussian Classifier
nb = GaussianNB()
nb_model = nb.fit(tfidf_train, targets_train )

# Predict Output
predictions = nb_model.predict(tfidf_train)

print( confusion_matrix(targets_train , predictions  ) )
print( accuracy_score(targets_train ,predictions) )

[[321   5   2   0   4   0   0   0   0   2   0   1   1]
 [  0 275   4   0   2   2   1   0   0   0   0   1   3]
 [  0   0 108   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0  47   0   0   0   0   0   0   0   0   0]
 [  0   0   2   0 142   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0  58   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0  28   0   0   0   0   0   0]
 [  9  11   7   0  13   2   0 544   0   6   1   4   2]
 [  8  14   2   1  10   4   0  10 726   2   0   6   5]
 [  0   0   0   0   0   2   1   0   0  86   1   0   0]
 [  0   0   0   0   0   0   1   0   0   0  54   0   0]
 [  0   0   0   1   1   0   0   0   0   0   0 159   0]
 [  0   0   0   1   0   0   0   0   0   0   1   0 184]]
0.945983379501385


In [11]:
# testData = count_vectorizer.transform(testData)
# tfidf_test_data = tfidf_transformer.transform(testData)

x_test = vec.transform(features_test).toarray()

# Predict Output
predictions = nb_model.predict(x_test)

print( confusion_matrix(targets_test , predictions  ) )
print( accuracy_score(targets_test ,predictions) )

[[ 30   5   2   1   4   1   0  36  56   3   0   3   4]
 [  9  21   1   0   3   1   0  24  57   1   0   2  11]
 [  9   6  10   0   1   0   0  13  18   0   0   1   2]
 [  1   2   1   0   0   0   0   0  15   0   0   0   1]
 [  4   1   0   1   7   1   0  15  13   0   0   1   0]
 [  2   1   1   0   0   0   1   2  17   0   1   3   0]
 [  1   0   0   0   0   0   4   4   4   0   0   0   0]
 [ 18  11   4   1   5   1   0 119  67   4   1  12   4]
 [ 29  18   4   0   4   2   1  46 200   3   0   8   2]
 [  3   0   0   0   1   0   0  16  10   1   0   0   0]
 [  4   0   0   0   2   0   1  19   8   0   0   1   1]
 [ 13   2   0   0   3   0   0  27  31   2   0   4   2]
 [  1  11   0   0   1   0   1  12  35   0   0   3  20]]
0.3360258481421648


## Read in texts to be annotated from TXT folder

The folder named 'TXT' contains transcripts of speeches delivered during the period 1970-2020.  

The code below extracts the sentences containing the term 'sovereign'.

In [12]:
dir = 'TXT'

txt_files = []

for root,dirs,files in os.walk( dir ):
    for file_name in files:
        
        if re.search( r'txt$' , file_name , re.IGNORECASE ) and not( re.search( r'^\.' , file_name , re.IGNORECASE ) ):
            txt_files.append( os.path.join(root, file_name) )

quotations = []

column_names = ['file_name','country','year','text','orig_text']
#sentences containing the word 'sovereign saved in df_result'
df_result = pd.DataFrame( columns = column_names )

quote_index = 0 
            
for file_name in txt_files:
    print(file_name)

    base_name = os.path.basename( file_name )
    base_name = re.sub( r'\.txt$' , '' , base_name )
    parts = re.split( r'_' , base_name )
    country = parts[0].strip()
    

    year = parts[2].strip()
    if re.search( r'_' , year ):
        year = year[ year.index('_')+1:]
    year = int(year)
    
    if not isinstance(year, int):
        print("year is an integer")
    

    with open( file_name , encoding = 'utf-8' ) as fh:
        #print(file_name)
        row = []
        speech = fh.read()
        #speech = clean_text(speech)
        sentences = sent_tokenize(speech)

        for quote in sentences:
            orig_quote = quote
            
            quote = re.sub( r'\s+' , ' ' , quote )
            if re.search( r'sovereign' , quote , re.IGNORECASE ):
                hits = re.findall( r'sovereign' , quote , re.IGNORECASE )

                quote_index += 1
                quote = re.sub( r',' , '' , quote )  
                quote = clean_text(quote)
                quote = lemmatise(quote)
                file_name_base = file_name[ file_name.index('/')+1 :]
                df_result.loc[quote_index] = [file_name_base,country,year,quote,orig_quote]
   

TXT/Session 73 - 2018/BRB_73_2018.txt
TXT/Session 73 - 2018/IND_73_2018.txt
TXT/Session 73 - 2018/ARG_73_2018.txt
TXT/Session 73 - 2018/JOR_73_2018.txt
TXT/Session 73 - 2018/SWE_73_2018.txt
TXT/Session 73 - 2018/EST_73_2018.txt
TXT/Session 73 - 2018/MDG_73_2018.txt
TXT/Session 73 - 2018/BTN_73_2018.txt
TXT/Session 73 - 2018/NOR_73_2018.txt
TXT/Session 73 - 2018/TUN_73_2018.txt
TXT/Session 73 - 2018/NRU_73_2018.txt
TXT/Session 73 - 2018/SWZ_73_2018.txt
TXT/Session 73 - 2018/ALB_73_2018.txt
TXT/Session 73 - 2018/IDN_73_2018.txt
TXT/Session 73 - 2018/BOL_73_2018.txt
TXT/Session 73 - 2018/SRB_73_2018.txt
TXT/Session 73 - 2018/GNQ_73_2018.txt
TXT/Session 73 - 2018/SYR_73_2018.txt
TXT/Session 73 - 2018/RUS_73_2018.txt
TXT/Session 73 - 2018/DJI_73_2018.txt
TXT/Session 73 - 2018/GIN_73_2018.txt
TXT/Session 73 - 2018/BHS_73_2018.txt
TXT/Session 73 - 2018/DNK_73_2018.txt
TXT/Session 73 - 2018/MYS_73_2018.txt
TXT/Session 73 - 2018/BRN_73_2018.txt
TXT/Session 73 - 2018/BGD_73_2018.txt
TXT/Session 

TXT/Session 29 - 1974/HUN_29_1974.txt
TXT/Session 29 - 1974/ISL_29_1974.txt
TXT/Session 29 - 1974/YUG_29_1974.txt
TXT/Session 29 - 1974/GRC_29_1974.txt
TXT/Session 29 - 1974/VEN_29_1974.txt
TXT/Session 29 - 1974/COD_29_1974.txt
TXT/Session 29 - 1974/CYP_29_1974.txt
TXT/Session 29 - 1974/COG_29_1974.txt
TXT/Session 29 - 1974/AUT_29_1974.txt
TXT/Session 29 - 1974/MEX_29_1974.txt
TXT/Session 29 - 1974/MYS_29_1974.txt
TXT/Session 29 - 1974/BGD_29_1974.txt
TXT/Session 29 - 1974/DDR_29_1974.txt
TXT/Session 29 - 1974/CAF_29_1974.txt
TXT/Session 29 - 1974/BGR_29_1974.txt
TXT/Session 29 - 1974/GBR_29_1974.txt
TXT/Session 29 - 1974/CHN_29_1974.txt
TXT/Session 29 - 1974/BFA_29_1974.txt
TXT/Session 29 - 1974/PRT_29_1974.txt
TXT/Session 29 - 1974/CRI_29_1974.txt
TXT/Session 29 - 1974/ECU_29_1974.txt
TXT/Session 29 - 1974/DZA_29_1974.txt
TXT/Session 29 - 1974/PHL_29_1974.txt
TXT/Session 29 - 1974/LKA_29_1974.txt
TXT/Session 29 - 1974/POL_29_1974.txt
TXT/Session 29 - 1974/BTN_29_1974.txt
TXT/Session 

TXT/Session 50 - 1995/ITA_50_1995.txt
TXT/Session 50 - 1995/ARE_50_1995.txt
TXT/Session 50 - 1995/LVA_50_1995.txt
TXT/Session 50 - 1995/TCD_50_1995.txt
TXT/Session 50 - 1995/PNG_50_1995.txt
TXT/Session 50 - 1995/NIC_50_1995.txt
TXT/Session 50 - 1995/SVK_50_1995.txt
TXT/Session 50 - 1995/FJI_50_1995.txt
TXT/Session 50 - 1995/TZA_50_1995.txt
TXT/Session 50 - 1995/BIH_50_1995.txt
TXT/Session 50 - 1995/KWT_50_1995.txt
TXT/Session 50 - 1995/NAM_50_1995.txt
TXT/Session 50 - 1995/IRN_50_1995.txt
TXT/Session 50 - 1995/SEN_50_1995.txt
TXT/Session 50 - 1995/NGA_50_1995.txt
TXT/Session 50 - 1995/IRQ_50_1995.txt
TXT/Session 50 - 1995/MLT_50_1995.txt
TXT/Session 50 - 1995/YEM_50_1995.txt
TXT/Session 50 - 1995/JAM_50_1995.txt
TXT/Session 50 - 1995/PRY_50_1995.txt
TXT/Session 50 - 1995/THA_50_1995.txt
TXT/Session 50 - 1995/HRV_50_1995.txt
TXT/Session 50 - 1995/GAB_50_1995.txt
TXT/Session 50 - 1995/MCO_50_1995.txt
TXT/Session 50 - 1995/BHR_50_1995.txt
TXT/Session 50 - 1995/AND_50_1995.txt
TXT/Session 

TXT/Session 64 - 2009/TON_64_2009.txt
TXT/Session 64 - 2009/CHL_64_2009.txt
TXT/Session 64 - 2009/DEU_64_2009.txt
TXT/Session 64 - 2009/COL_64_2009.txt
TXT/Session 64 - 2009/TLS_64_2009.txt
TXT/Session 64 - 2009/PSE_64_2009.txt
TXT/Session 64 - 2009/PER_64_2009.txt
TXT/Session 64 - 2009/SYC_64_2009.txt
TXT/Session 64 - 2009/LTU_64_2009.txt
TXT/Session 64 - 2009/SVN_64_2009.txt
TXT/Session 64 - 2009/KEN_64_2009.txt
TXT/Session 64 - 2009/MDV_64_2009.txt
TXT/Session 64 - 2009/KHM_64_2009.txt
TXT/Session 64 - 2009/VUT_64_2009.txt
TXT/Session 64 - 2009/KNA_64_2009.txt
TXT/Session 64 - 2009/ISR_64_2009.txt
TXT/Session 64 - 2009/ERI_64_2009.txt
TXT/Session 64 - 2009/CUB_64_2009.txt
TXT/Session 64 - 2009/CHE_64_2009.txt
TXT/Session 64 - 2009/HND_64_2009.txt
TXT/Session 64 - 2009/MKD_64_2009.txt
TXT/Session 64 - 2009/CIV_64_2009.txt
TXT/Session 64 - 2009/SGP_64_2009.txt
TXT/Session 64 - 2009/KOR_64_2009.txt
TXT/Session 51 - 1996/RWA_51_1996.txt
TXT/Session 51 - 1996/JPN_51_1996.txt
TXT/Session 

TXT/Session 27 - 1972/UKR_27_1972.txt
TXT/Session 27 - 1972/ROU_27_1972.txt
TXT/Session 27 - 1972/MWI_27_1972.txt
TXT/Session 27 - 1972/BEN_27_1972.txt
TXT/Session 27 - 1972/GAB_27_1972.txt
TXT/Session 27 - 1972/BHR_27_1972.txt
TXT/Session 27 - 1972/PRY_27_1972.txt
TXT/Session 27 - 1972/THA_27_1972.txt
TXT/Session 27 - 1972/MLT_27_1972.txt
TXT/Session 27 - 1972/IRQ_27_1972.txt
TXT/Session 27 - 1972/JAM_27_1972.txt
TXT/Session 27 - 1972/SOM_27_1972.txt
TXT/Session 27 - 1972/YEM_27_1972.txt
TXT/Session 27 - 1972/NGA_27_1972.txt
TXT/Session 27 - 1972/SEN_27_1972.txt
TXT/Session 27 - 1972/IRN_27_1972.txt
TXT/Session 27 - 1972/YDYE_27_1972.txt
TXT/Session 27 - 1972/KWT_27_1972.txt
TXT/Session 27 - 1972/TZA_27_1972.txt
TXT/Session 27 - 1972/FJI_27_1972.txt
TXT/Session 27 - 1972/NIC_27_1972.txt
TXT/Session 27 - 1972/ITA_27_1972.txt
TXT/Session 27 - 1972/TCD_27_1972.txt
TXT/Session 27 - 1972/ARE_27_1972.txt
TXT/Session 27 - 1972/MMR_27_1972.txt
TXT/Session 27 - 1972/TGO_27_1972.txt
TXT/Session

...

TXT/Session 43 - 1988/ARE_43_1988.txt
TXT/Session 43 - 1988/BLZ_43_1988.txt
TXT/Session 43 - 1988/LSO_43_1988.txt
TXT/Session 43 - 1988/TZA_43_1988.txt
TXT/Session 43 - 1988/FJI_43_1988.txt
TXT/Session 43 - 1988/PNG_43_1988.txt
TXT/Session 43 - 1988/NIC_43_1988.txt
TXT/Session 43 - 1988/EGY_43_1988.txt
TXT/Session 43 - 1988/NZL_43_1988.txt
TXT/Session 43 - 1988/CPV_43_1988.txt
TXT/Session 43 - 1988/UGA_43_1988.txt
TXT/Session 43 - 1988/CSK_43_1988.txt
TXT/Session 43 - 1988/TGO_43_1988.txt
TXT/Session 43 - 1988/MMR_43_1988.txt
TXT/Session 43 - 1988/NLD_43_1988.txt
TXT/Session 43 - 1988/AUS_43_1988.txt
TXT/Session 43 - 1988/LBN_43_1988.txt
TXT/Session 43 - 1988/SLE_43_1988.txt
TXT/Session 43 - 1988/MDV_43_1988.txt
TXT/Session 43 - 1988/KHM_43_1988.txt
TXT/Session 43 - 1988/KEN_43_1988.txt
TXT/Session 43 - 1988/SGP_43_1988.txt
TXT/Session 43 - 1988/HND_43_1988.txt
TXT/Session 43 - 1988/CUB_43_1988.txt
TXT/Session 43 - 1988/VUT_43_1988.txt
TXT/Session 43 - 1988/KNA_43_1988.txt
TXT/Session 

TXT/Session 61 - 2006/GMB_61_2006.txt
TXT/Session 61 - 2006/SLV_61_2006.txt
TXT/Session 61 - 2006/TLS_61_2006.txt
TXT/Session 61 - 2006/COL_61_2006.txt
TXT/Session 61 - 2006/DEU_61_2006.txt
TXT/Session 61 - 2006/CHL_61_2006.txt
TXT/Session 61 - 2006/TON_61_2006.txt
TXT/Session 61 - 2006/LTU_61_2006.txt
TXT/Session 61 - 2006/SVN_61_2006.txt
TXT/Session 61 - 2006/SYC_61_2006.txt
TXT/Session 61 - 2006/PSE_61_2006.txt
TXT/Session 61 - 2006/PER_61_2006.txt
TXT/Session 61 - 2006/LSO_61_2006.txt
TXT/Session 61 - 2006/BLZ_61_2006.txt
TXT/Session 61 - 2006/KIR_61_2006.txt
TXT/Session 61 - 2006/ITA_61_2006.txt
TXT/Session 61 - 2006/AGO_61_2006.txt
TXT/Session 61 - 2006/ARE_61_2006.txt
TXT/Session 61 - 2006/LVA_61_2006.txt
TXT/Session 61 - 2006/TCD_61_2006.txt
TXT/Session 61 - 2006/PNG_61_2006.txt
TXT/Session 61 - 2006/NIC_61_2006.txt
TXT/Session 61 - 2006/SVK_61_2006.txt
TXT/Session 61 - 2006/FJI_61_2006.txt
TXT/Session 61 - 2006/TZA_61_2006.txt
TXT/Session 61 - 2006/UGA_61_2006.txt
TXT/Session 

TXT/Session 54 - 1999/ALB_54_1999.txt
TXT/Session 54 - 1999/BTN_54_1999.txt
TXT/Session 54 - 1999/NOR_54_1999.txt
TXT/Session 54 - 1999/TUN_54_1999.txt
TXT/Session 54 - 1999/EST_54_1999.txt
TXT/Session 54 - 1999/MDG_54_1999.txt
TXT/Session 54 - 1999/DZA_54_1999.txt
TXT/Session 54 - 1999/PRK_54_1999.txt
TXT/Session 54 - 1999/LKA_54_1999.txt
TXT/Session 54 - 1999/POL_54_1999.txt
TXT/Session 54 - 1999/PHL_54_1999.txt
TXT/Session 54 - 1999/GNB_54_1999.txt
TXT/Session 54 - 1999/BGR_54_1999.txt
TXT/Session 54 - 1999/GBR_54_1999.txt
TXT/Session 54 - 1999/BFA_54_1999.txt
TXT/Session 54 - 1999/PRT_54_1999.txt
TXT/Session 54 - 1999/CRI_54_1999.txt
TXT/Session 54 - 1999/ECU_54_1999.txt
TXT/Session 54 - 1999/CHN_54_1999.txt
TXT/Session 54 - 1999/BGD_54_1999.txt
TXT/Session 54 - 1999/MYS_54_1999.txt
TXT/Session 54 - 1999/BRN_54_1999.txt
TXT/Session 54 - 1999/SLB_54_1999.txt
TXT/Session 54 - 1999/TJK_54_1999.txt
TXT/Session 54 - 1999/ATG_54_1999.txt
TXT/Session 54 - 1999/AUT_54_1999.txt
TXT/Session 

TXT/Session 47 - 1992/NZL_47_1992.txt
TXT/Session 47 - 1992/PNG_47_1992.txt
TXT/Session 47 - 1992/NIC_47_1992.txt
TXT/Session 47 - 1992/TZA_47_1992.txt
TXT/Session 47 - 1992/FJI_47_1992.txt
TXT/Session 47 - 1992/BLZ_47_1992.txt
TXT/Session 47 - 1992/LSO_47_1992.txt
TXT/Session 47 - 1992/AGO_47_1992.txt
TXT/Session 47 - 1992/ITA_47_1992.txt
TXT/Session 47 - 1992/TCD_47_1992.txt
TXT/Session 47 - 1992/LVA_47_1992.txt
TXT/Session 47 - 1992/ARE_47_1992.txt
TXT/Session 47 - 1992/DMA_47_1992.txt
TXT/Session 47 - 1992/NGA_47_1992.txt
TXT/Session 47 - 1992/SEN_47_1992.txt
TXT/Session 47 - 1992/MLT_47_1992.txt
TXT/Session 47 - 1992/IRQ_47_1992.txt
TXT/Session 47 - 1992/JAM_47_1992.txt
TXT/Session 47 - 1992/YEM_47_1992.txt
TXT/Session 47 - 1992/KWT_47_1992.txt
TXT/Session 47 - 1992/BIH_47_1992.txt
TXT/Session 47 - 1992/IRN_47_1992.txt
TXT/Session 47 - 1992/NAM_47_1992.txt
TXT/Session 47 - 1992/STP_47_1992.txt
TXT/Session 47 - 1992/ROU_47_1992.txt
TXT/Session 47 - 1992/GRD_47_1992.txt
TXT/Session 

TXT/Session 69 - 2014/LIE_69_2014.txt
TXT/Session 69 - 2014/AZE_69_2014.txt
TXT/Session 69 - 2014/GRC_69_2014.txt
TXT/Session 69 - 2014/ISL_69_2014.txt
TXT/Session 69 - 2014/HUN_69_2014.txt


In [13]:
df_result.to_csv('un_speeches_uncategorised.csv')
print( f'{df_result.shape[0]} sentences containing the term \'sovereign\' ')

12952 sentences containing the term 'sovereign' 


Use the trained model to classify the sentences

In [14]:

x_new = vec.transform(df_result['text']).toarray()

# Predict Output
predictions = knn_model.predict(x_new)
df_result['category'] = predictions


In [15]:
print(df_result.columns)
df_result['year'] = df_result['year'].astype('int')
print(df_result.dtypes)

df_result.to_csv('un_speeches.csv')

Index(['file_name', 'country', 'year', 'text', 'orig_text', 'category'], dtype='object')
file_name    object
country      object
year          int64
text         object
orig_text    object
category     object
dtype: object


In [16]:
df_result['date'] = pd.to_datetime( df_result['year'] )

Make CSV files for each decade

In [17]:
import datetime
this_year = datetime.date.today().year


for year in range(1970,this_year,10):
    print(year)
    df_decade = df_result.query( 'date >= {} and date < {}'.format(year,year+10))
    outfile = f'speeches_{year}s.tsv'
    if df_decade.shape[0] > 0:
        df_decade.drop(columns=['date', 'text'])
        df_decade.to_csv(outfile) 


1970
1980
1990
2000
2010
2020
